In [1]:
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#### TODO
- Separate notebooks into training and inference
- Create a config file for training

In [2]:
# Define hyperparameters
# Sequence length also acts similarly to the warm up period in conventional models
seq_length = 730
# Portion of data for training, from which test proportion is inferred
train_split = 0.8
# Batch size should be a exponent of base 2
batch_size = 256
# Hidden size of LSTM
hidden_size = 50
# Number of stacked layers of LSTM
num_layers = 4
# Initial learning rate
lr = 0.001
# Number of epochs
epochs = 10
# Embedding size
embedding_size = 8
# # Patience threshold of learning rate scheduler
patience = 10

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# Function to load data
def load_data(file_path, prefix):
    return [
        pd.read_csv(p, parse_dates=["Date"], dayfirst=True)
        for p in Path(file_path).glob(f"{prefix}*.csv")
    ]

In [4]:
# Load in observed groundwater and meteorological data
df_gwl = pd.concat(load_data("../data_processed/", "AquiMod_"))
df_met = pd.concat(load_data("../data_processed/", "ukcp18_"))
df_gwl['borehole_id'] = (df_gwl['Borehole'] != df_gwl['Borehole'].shift()).cumsum() - 1

In [5]:
df_data = pd.merge(left=df_gwl, right=df_met, on=["Borehole", "Model", "Date"], how="inner").dropna()
df_data

,Borehole,Model,Date,Sim,Obs,borehole_id,precipwsnow,PET
16334,Allington No 2,AquiMod,2006-09-21,72.5336,67.997000,0,0.156802,2.210
16335,Allington No 2,AquiMod,2006-09-22,72.4902,67.933000,0,8.686279,2.210
16336,Allington No 2,AquiMod,2006-09-23,72.4493,67.903000,0,12.740111,2.210
16337,Allington No 2,AquiMod,2006-09-24,72.4086,67.859000,0,0.284288,2.210
16338,Allington No 2,AquiMod,2006-09-25,72.3679,67.754000,0,0.002568,2.210
...,...,...,...,...,...,...,...,...
1119157,Woodend Farm,AquiMod,2005-02-14,85.1202,86.970968,53,0.002877,0.675
1119158,Woodend Farm,AquiMod,2005-02-15,85.1124,86.983226,53,0.000412,0.675
1119159,Woodend Farm,AquiMod,2005-02-16,85.1042,86.995484,53,0.008233,0.675
1119160,Woodend Farm,AquiMod,2005-02-17,85.0955,87.007742,53,0.470635,0.675


In [6]:
precip_scaler = StandardScaler()
pet_scaler = StandardScaler()
gwl_scalers = [StandardScaler()] * (df_data["borehole_id"].max() + 1)

The normalisation should occur on the calibration data only, not the validation.

In [7]:
precip = precip_scaler.fit_transform(df_data["precipwsnow"].values.reshape(-1, 1))
pet = pet_scaler.fit_transform(df_data["PET"].values.reshape(-1, 1))
bhid = df_data["borehole_id"].values.reshape(-1, 1)
gwl = np.empty((0, 1))
for i in df_data["borehole_id"].unique():
    gwl = np.append(
        gwl,
        gwl_scalers[i].fit_transform(
            df_data.query("borehole_id == @i")["Obs"].values.reshape(-1, 1)
        ),
        axis=0
    )

In [8]:
def create_sequences(data: np.ndarray, seq_length: int) -> np.ndarray:
    """
    Transforms 2D time-series data into an array of sequences of a specified length.

    Parameters:
    data (np.ndarray): A 2D numpy array where each row is a time step and each column is a feature.
    seq_length (int): The number of time steps to include in each output sequence.

    Returns:
    np.ndarray: A 3D numpy array of shape (num_samples - seq_length + 1, seq_length, num_features).
    """

    xs = []  # Initialise an empty list to store sequences

    # For each possible sequence in the data...
    for i in range(len(data) - seq_length + 1):
        # Extract a sequence of length `seq_length`
        x = data[i: (i + seq_length)]
        # Append the sequence to the list
        xs.append(x)

    # Convert the list of sequences into a 3D numpy array
    return np.array(xs)

In [9]:
# create_sequences has to be called individually on each timeseries from each borehole
df_scaled = pd.DataFrame(np.column_stack((precip, pet, bhid, gwl)), columns=["precip", "pet", "bhid", "gwl"])
df_scaled["bhid"] = df_scaled["bhid"].astype(int)

# Initialise lists to hold dynamic and static data for each borehole for train and test periods
dynamic_train_list = []
dynamic_test_list = []
static_train_list = []
static_test_list = []
gwl_train_list = []
gwl_test_list = []
# For each borehole
for i in np.unique(bhid):
    # Extract data for each borehole and reshape into 2D
    _precip = df_scaled.query("bhid == @i")["precip"].values.reshape(-1, 1)
    _pet = df_scaled.query("bhid == @i")["pet"].values.reshape(-1, 1)
    _bhid = df_scaled.query("bhid == @i")["bhid"].values.reshape(-1, 1)
    _gwl = df_scaled.query("bhid == @i")["gwl"].values.reshape(-1, 1)[seq_length - 1:]
    # Convert 2D timeseries to a 3D timeseries of sequences
    dynamic = create_sequences(np.column_stack((_precip, _pet)), seq_length)
    static = create_sequences(_bhid, seq_length)

    assert len(dynamic) == len(static)

    train_size = int(len(dynamic) * train_split)
    test_size = len(dynamic) - train_size

    dynamic_train_list.append(dynamic[:train_size])
    dynamic_test_list.append(dynamic[train_size:])
    static_train_list.append(static[:train_size])
    static_test_list.append(static[train_size:])
    gwl_train_list.append(_gwl[:train_size])
    gwl_test_list.append(_gwl[train_size:])


dynamic_train_arr = np.concatenate(dynamic_train_list)
dynamic_test_arr = np.concatenate(dynamic_test_list)
static_train_arr = np.concatenate(static_train_list)
static_test_arr = np.concatenate(static_test_list)
gwl_train_arr = np.concatenate(gwl_train_list)
gwl_test_arr = np.concatenate(gwl_test_list)

In [10]:
print(dynamic_train_arr.shape)
print(dynamic_test_arr.shape)
print(static_train_arr.shape)
print(static_test_arr.shape)
print(gwl_train_arr.shape)
print(gwl_test_arr.shape)

(628785, 730, 2)
(157227, 730, 2)
(628785, 730, 1)
(157227, 730, 1)
(628785, 1)
(157227, 1)


In [11]:
# Save ram, delete the lists
del dynamic_train_list
del dynamic_test_list
del static_train_list
del static_test_list
del gwl_train_list
del gwl_test_list

I need to make sure that the catchment_ids are arranged in the same way with the first two dimensions of dynamic_input_arr.

Also, because this data is getting large, maybe I need to incorporate this create_sequences functionality into the dataset or dataloader so that sequences are only created when they are passed?

In [12]:
dynamic_train_tensor = torch.from_numpy(dynamic_train_arr).float()
dynamic_test_tensor = torch.from_numpy(dynamic_test_arr).float()
static_train_tensor = torch.from_numpy(static_train_arr)
static_test_tensor = torch.from_numpy(static_test_arr)
gwl_train_tensor = torch.from_numpy(gwl_train_arr).float()
gwl_test_tensor = torch.from_numpy(gwl_test_arr).float()

# Make sure the static layer is 2D (because embeddings layer expects a 2D tensor)
static_train_tensor = static_train_tensor.view(
    static_train_tensor.size(0), static_train_tensor.size(1)
)
static_test_tensor = static_test_tensor.view(
    static_test_tensor.size(0), static_test_tensor.size(1)
)

In [13]:
print(dynamic_train_tensor.dtype)
print(dynamic_test_tensor.dtype)
print(static_train_tensor.dtype)
print(static_test_tensor.dtype)
print(gwl_train_tensor.dtype)
print(gwl_test_tensor.dtype)

torch.float32
torch.float32
torch.int32
torch.int32
torch.float32
torch.float32


In [14]:
print(dynamic_train_tensor.shape)
print(dynamic_test_tensor.shape)
print(static_train_tensor.shape)
print(static_test_tensor.shape)
print(gwl_train_tensor.shape)
print(gwl_test_tensor.shape)

torch.Size([628785, 730, 2])
torch.Size([157227, 730, 2])
torch.Size([628785, 730])
torch.Size([157227, 730])
torch.Size([628785, 1])
torch.Size([157227, 1])


In [15]:
# Define the dataset class
class MultiTimeSeriesDataset(Dataset):
    """Pytorch dataset class for timeseries sequences data with both dynamic and static features."""
    def __init__(
            self,
            dynamic: torch.Tensor,
            static: torch.Tensor,
            target: torch.Tensor
        ):
        self.dynamic = dynamic
        self.static = static
        self.target = target

    def __len__(self):
        return len(self.dynamic)

    def __getitem__(self, i):
        return self.dynamic[i], self.static[i], self.target[i]

In [16]:
# Instantiate datasets and dataloaders
train_dataset = MultiTimeSeriesDataset(dynamic_train_tensor, static_train_tensor, gwl_train_tensor)
test_dataset = MultiTimeSeriesDataset(dynamic_test_tensor, static_test_tensor, gwl_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

The `catchment_ids` tensor should have the same batch size as your input data `x`. If `x` is a 3D tensor of shape `(batch_size, sequence_length, num_features)`, then `catchment_ids` should be a 2D tensor of shape `(batch_size, sequence_length)`.

Each element in the `catchment_ids` tensor corresponds to the catchment identifier for the corresponding data point in `x`. When these identifiers are passed through the embedding layer, they are transformed into dense vectors (embeddings) of a fixed size (`embedding_size`). These embeddings are then concatenated with your other features (precipitation and PET), resulting in a tensor of shape `(batch_size, sequence_length, num_features + embedding_size)`.

Here's a visual representation of the process:

```
catchment_ids (batch_size, sequence_length)
       |
Embedding Layer
       |
catchment_embeddings (batch_size, sequence_length, embedding_size)
       |
Concatenation with x ---> input to LSTM (batch_size, sequence_length, num_features + embedding_size)
```

Remember, the catchment identifiers should be integers in the range `[0, num_catchments)`, where `num_catchments` is the number of unique catchments. The embeddings for these identifiers are learned during training. 😊

In [17]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(
            self,
            dynamic_size: int,
            static_len: int,
            embedding_size: int,
            hidden_size: int,
            num_layers: int,
            output_size: int,
        ):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(static_len, embedding_size, max_norm=1)
        self.lstm = nn.LSTM(dynamic_size + embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, dynamic: torch.Tensor, static: torch.Tensor):
        # Pass catchment identifiers through embedding layer
        static_embeddings = self.embedding(static)
        # Concatenate catchment embeddings with other features
        x = torch.cat((dynamic, static_embeddings), dim=-1)
        # Initialise hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Initialise cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate LSTM using input data, x, and initial states, h0 and c0
        # x is a batch of input sequences of features
        # h contains the updated hidden states from each timestep
        # c contains the update cell states from each timestep
        h, c = self.lstm(x, (h0, c0))
        # The last hidden state from the output sequence is pass to the fully connected layer
        out = self.fc(h[:, -1, :])
        return out

The catchment identifier for each output of your model should be known from your input data. When you prepare your batches of input data, you should keep track of the catchment identifier associated with each sequence in your batch. This is typically done by keeping the catchment identifiers in a separate tensor or list that aligns with your batch of input data.

Here's an example:

```python
# Assume catchment_ids is a tensor of your catchment identifiers
catchment_ids = torch.tensor([...])

# Assume x is your input data
x = torch.tensor([...])

# Pass catchment_ids and x through your model
output = model(x, catchment_ids)

# Now, the i-th element of output corresponds to the i-th catchment in catchment_ids
```

In this example, the i-th element of `output` corresponds to the i-th catchment in `catchment_ids`. So, you know which catchment each output belongs to.

Remember, the embeddings themselves don't contain information about the catchment identifiers. They are just a learned representation of the catchments that helps your model make better predictions. The actual mapping between catchment identifiers and catchment names should be maintained separately in your data preprocessing step. 😊

In [18]:
# # Define Nash-Sutcliffe Efficiency function for post-training analysis
# def nse(predictions: np.ndarray, targets: np.ndarray) -> float:
#     denominator = np.sum((targets - np.mean(targets)) ** 2)
#     numerator = np.sum((targets - predictions) ** 2)
#     nse_val = 1 - numerator / denominator
#     return nse_val

In [19]:
# Define the training function
def train_epoch(model: LSTM, criterion: nn.Module, optimiser: nn.Module):
    model.train()
    running_loss = 0
    for batch in train_loader:
        # batch is a list of three elements, the dynamic, static and the target
        dynamic_batch, static_batch, target_batch = (
            batch[0].to(device), batch[1].to(device), batch[2].to(device)
        )
        # Forward propagate the model and get outputs
        output = model(dynamic_batch, static_batch)
        # Calculate loss between outputs and the target
        loss = criterion(output, target_batch)
        # Add loss to the running loss
        running_loss += loss.item()
        # Reset the gradients
        optimiser.zero_grad()
        # This, rather unpythonically, computes the gradients for all model parameters
        loss.backward()
        # Take a step in the gradient direction
        optimiser.step()
    # Return the average loss for each batch across the epoch
    return running_loss / len(train_loader)

In [20]:
# Define the testing function
def test_epoch(model: LSTM, criterion: nn.Module):
    model.eval()
    running_loss = 0
    for batch in test_loader:
        # batch is a list of three elements, the dynamic, static and the target
        dynamic_batch, static_batch, target_batch = (
            batch[0].to(device), batch[1].to(device), batch[2].to(device)
        )
        with torch.no_grad():
            # Forward propagate the model and get outputs
            output = model(dynamic_batch, static_batch)
            # Calculate loss between outputs and the target
            loss = criterion(output, target_batch)
            # Add loss to the running loss
            running_loss += loss.item()
    # Return the average loss for each batch across the epoch
    return running_loss / len(test_loader)

In [21]:
# Initialise the model, loss function, and optimiser
model = LSTM(
    dynamic_size=dynamic_train_tensor.size()[-1],
    static_len=len(np.unique(bhid)),
    embedding_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    output_size=gwl_train_tensor.size()[-1],
).to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimiser, patience=patience, verbose=True)
criterion = nn.MSELoss()

In [22]:
# Initialise lists for plotting
train_loss = []
test_loss = []

In [122]:
for epoch in range(epochs):
    train_loss.append(train_epoch(model, criterion, optimiser))
    test_loss.append(test_epoch(model, criterion))
    scheduler.step(train_loss[-1])

    # Calculate NSE based on unscaled data
    print(f"Epoch {epoch + 1} Train: {round(train_loss[-1], 8)}, Test: {round(test_loss[-1], 8)}")

Epoch 1 Train: 0.42427439, Test: 0.19974023
Epoch 2 Train: 0.12766968, Test: 0.18103211
Epoch 3 Train: 0.09440576, Test: 0.18737793
Epoch 4 Train: 0.07252766, Test: 0.20581782
Epoch 5 Train: 0.05849689, Test: 0.20561467
Epoch 6 Train: 0.04449196, Test: 0.21526666
Epoch 7 Train: 0.03657804, Test: 0.21127147
Epoch 8 Train: 0.02997669, Test: 0.20272135
Epoch 9 Train: 0.02423411, Test: 0.20661022
Epoch 10 Train: 0.02098081, Test: 0.21583155


In [22]:
# torch.save(model, "temp_model.pt")

In [100]:
px.line(pd.DataFrame(np.column_stack((train_loss, test_loss))))

In [22]:
model = torch.load("temp_model.pt")

In [23]:
def inference(model, dynamic_sequence, static_data):
    """
    Perform inference with the trained LSTM model.

    Parameters:
    - model: Trained LSTM model
    - dynamic_sequence: Input dynamic sequence (shape: [sequence_length, dynamic_size])
    - static_data: Input static data (shape: [sequence_length])

    Returns:
    - predictions: Model predictions (shape: [sequence_length, output_size])
    """

    # Ensure that dynamic_sequence has the shape [batch_size, sequence_length, dynamic_size]
    # Ensure that static_data has the shape [batch_size, sequence_length]

    # Forward pass through the model
    with torch.no_grad():
        predictions = model(dynamic_sequence, static_data)

    return predictions.cpu().numpy()  # Convert predictions to a NumPy array


In [24]:
# Example usage:
# Assuming model is your trained LSTM model and dynamic_sequence, static_data are your input sequences
# dynamic sequence (shape: [timesteps, sequence_length, dynamic_size])
# static data (shape: [sequence_length])
# 3296
first_bh_len = 3000
dynamic_tensor = dynamic_train_tensor[:first_bh_len].to(device)
static_tensor = static_train_tensor[:first_bh_len].to(device)



In [25]:
predictions_example = inference(model, dynamic_tensor, static_tensor)
# predictions_real = gwl_scalers[0].inverse_transform(predictions_example)
# observed_real = gwl_scalers[0].inverse_transform(gwl[:first_bh_len])

In [27]:
px.line(pd.DataFrame(np.column_stack((gwl[seq_length:seq_length + first_bh_len], predictions_example))))

In [28]:
output_list = []
for batch in test_loader:
    # batch is a list of three elements, the dynamic, static and the target
    dynamic_batch, static_batch, target_batch = (
        batch[0].to(device), batch[1].to(device), batch[2].to(device)
    )
    # Forward propagate the model and get outputs
    output_list.append(inference(model, dynamic_batch, static_batch))

In [29]:
%matplotlib qt

In [30]:
fig, ax = plt.subplots(nrows=3, ncols=1)
ax[0].plot(np.column_stack((np.concatenate(output_list), gwl_test_tensor)))
ax[1].plot(df_data.query("borehole_id == 53")["Obs"].values)
ax[2].plot(gwl_scalers[53].transform(df_data.query("borehole_id == 53")["Obs"].values.reshape(-1, 1)))
plt.plot()

[]

In [33]:
plt.plot(np.column_stack((gwl_test_tensor, np.concatenate(output_list))))
plt.show()

In [34]:
pd.DataFrame(np.column_stack((gwl_test_tensor, np.concatenate(output_list)))).to_csv("temp.csv")

Right now I am scaling each borehole individually. Is this the correct thing to do?
Also, I should separate out training and inference.

Also, I am thinking about the fintetuning model. When finetuning a model on an individual borehole, should I scale the individual well to amplitude less than 2 (-1, 1)? This way, it will probably do better at modelling extremes greater and lesser than in the training data. I would need to test the relationship between inputs (extreme precipitation event) and how well this can predict out GWL

In [28]:
def count_parameters(model: LSTM) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model)} parameters")

The model has 119793 parameters
